In [31]:
import numpy as np
import matplotlib.pyplot as plt
import os
import math
import shutil
import glob

In [32]:
ROOT_DIR="./Datasets/lung/"
number_of_images={}
for dir in os.listdir(ROOT_DIR):
    number_of_images[dir]=len(os.listdir(os.path.join(ROOT_DIR,dir)))
print(number_of_images)
classes=list(number_of_images.keys())

{'Normal cases': 416, 'Malignant cases': 561, 'Bengin cases': 120}


In [33]:
def datafolder(path,split):
    if not os.path.exists('./'+path):
        os.mkdir('./'+path)
        for dir in os.listdir(ROOT_DIR):
            os.makedirs("./"+path+"/"+dir)
            for img in np.random.choice(a=os.listdir(os.path.join(ROOT_DIR,dir)),size=(math.floor(split*number_of_images[dir])),replace=False):
                O=os.path.join(ROOT_DIR,dir,img)
                D=os.path.join("./"+path+"/",dir)
                shutil.copy(O,D)
    else :
        print(path+" Folder exists")

In [34]:
datafolder("train",0.7)
datafolder("val",0.5)
datafolder("test",1)

train Folder exists
val Folder exists
test Folder exists


In [35]:
from keras.models import Sequential
from keras.layers import Conv2D,MaxPool2D,Dropout,Flatten,Dense,BatchNormalization,GlobalAvgPool2D
from keras.preprocessing.image import load_img,img_to_array,ImageDataGenerator
import keras

In [36]:
model=Sequential()
model.add(Conv2D(filters= 8,kernel_size=(3,3),activation='relu',input_shape=(512,512,3),padding='same'))
model.add(Conv2D(filters= 16,kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(filters= 32,kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(filters=64,kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(rate=0.25))
model.add(Flatten())
model.add(Dense(units=64,activation='relu'))
model.add(Dropout(rate=0.25))
model.add(Dense(3))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 512, 512, 8)       224       
                                                                 
 conv2d_5 (Conv2D)           (None, 510, 510, 16)      1168      
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 255, 255, 16)      0         
 g2D)                                                            
                                                                 
 conv2d_6 (Conv2D)           (None, 253, 253, 32)      4640      
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 126, 126, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_7 (Conv2D)           (None, 124, 124, 64)     

In [37]:
model.compile(optimizer='adam',loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])

In [38]:
train_data=ImageDataGenerator(zoom_range=0.2,shear_range=0.2,rescale=1/255,horizontal_flip=True).flow_from_directory(directory='./train',target_size=(512,512),batch_size=32,class_mode='binary')
val_data=ImageDataGenerator(rescale=1/255).flow_from_directory(directory='./val',target_size=(512,512),batch_size=32,class_mode='binary')
test_data=ImageDataGenerator(rescale=1/255).flow_from_directory(directory='./test',target_size=(512,512),batch_size=32,class_mode='binary')

Found 767 images belonging to 3 classes.
Found 548 images belonging to 3 classes.
Found 1097 images belonging to 3 classes.


In [39]:
from keras.callbacks import ModelCheckpoint,EarlyStopping
es=EarlyStopping(monitor='val_accuracy',min_delta=0.01,patience=3,verbose=1,mode='auto') # type: ignore
M=ModelCheckpoint(monitor='val_accuracy',filepath='./best.h5',verbose=1,save_best_only=True,mode='auto')
cd=[es,M]

In [40]:
hs=model.fit(train_data,steps_per_epoch=8,epochs=10,verbose=1,validation_data=val_data,validation_steps=16,callbacks=cd) # type: ignore

Epoch 1/10
8/8 [==============================] - ETA: 0s - loss: 2.0665 - accuracy: 0.3828
Epoch 1: val_accuracy improved from -inf to 0.71680, saving model to ./best.h5


/home/share/Machine Learning/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


8/8 [==============================] - 32s 4s/step - loss: 2.0665 - accuracy: 0.3828 - val_loss: 0.9277 - val_accuracy: 0.7168
Epoch 2/10
8/8 [==============================] - ETA: 0s - loss: 0.9220 - accuracy: 0.5412
Epoch 2: val_accuracy did not improve from 0.71680
8/8 [==============================] - 30s 4s/step - loss: 0.9220 - accuracy: 0.5412 - val_loss: 0.8490 - val_accuracy: 0.6152
Epoch 3/10
8/8 [==============================] - ETA: 0s - loss: 0.8520 - accuracy: 0.6289
Epoch 3: val_accuracy improved from 0.71680 to 0.74219, saving model to ./best.h5
8/8 [==============================] - 31s 4s/step - loss: 0.8520 - accuracy: 0.6289 - val_loss: 0.7799 - val_accuracy: 0.7422
Epoch 4/10
8/8 [==============================] - ETA: 0s - loss: 0.8455 - accuracy: 0.6549
Epoch 4: val_accuracy did not improve from 0.74219
8/8 [==============================] - 30s 4s/step - loss: 0.8455 - accuracy: 0.6549 - val_loss: 0.7872 - val_accuracy: 0.5742
Epoch 5/10
8/8 [================

In [ ]:
img=load_img('Datasets/lung/Malignant cases/Malignant case (73).jpg',target_size=(512,512))
arr=img_to_array(img)
arr=np.expand_dims(arr,axis=0)
print(arr.shape)
pred=model.predict(arr)[0]
print(pred)
print(classes[pred.index(max(pred))])

(1, 512, 512, 3)
1/1 [==============================] - 0s 32ms/step


[-206.80734  203.39685  185.69939]


NameError: name 'classes' is not defined